#### Imports

In [ ]:
## llibrerias
from pathlib import Path
import pandas as pd
from glob import glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os
import imageio
from IPython.display import Image

## Funcions propies
from src_preproces.psychopy_extractor.parser import create_all_sub_folder
from src_preproces.biosignalsplux.biosignalsProcessor import open_signals_npy
from src_preproces.rPPG_extraction.rPPG_POS_method import rPPG_detector

#### Inicialització

In [ ]:
## Paths
path_init = Path("D:/TFG_ALEX/")
path_infos = path_init / "cleaned_data"
path_patients = path_init / "raw_data/recordings"
path_img_save = Path("../imatges") ## carpeta on guardar imatges

In [ ]:
create_all_sub_folder(
    psychopy_folder = path_init / "raw_data/psychopy_annotations/data",
    destiny_folder = path_infos    
    )

### Analicis dels subjectes

#### Crear a pd amb la informacio de cada subjecte

In [ ]:
name_file = "demographic.csv"
subs_folder = glob("sub_*", root_dir=path_infos)
#print(subs_folder)

subject_df = pd.read_csv(path_infos / subs_folder[0] / name_file)
for sub in subs_folder[1:]:
    subject_df2 = pd.read_csv(path_infos / sub / name_file)
    subject_df = pd.concat([subject_df, subject_df2], ignore_index = True)

subject_df = subject_df.sort_values(by = "subject_id")
subject_df = subject_df.reset_index(drop = True)
subject_df

#### Genera

In [ ]:
gender = subject_df["gender"].unique()
count_g = [len(subject_df[subject_df["gender"] == x]) for x in gender]
clr_g = ['orange','dodgerblue','green','grey']

fig, ax = plt.subplots()
ax.bar(gender, count_g, color=clr_g)
ax.set_yticklabels([])
for i,n in enumerate(count_g):
    ax.text( i , n, n, ha='center', va='bottom')
plt.show()

#### Age

In [ ]:
age = np.arange(subject_df["age"].min(), subject_df["age"].max()+1)
count_a = [len(subject_df[subject_df["age"] == x]) for x in age]
#print(age)
#print(count_a)

fig, ax = plt.subplots()
ax.bar(age, count_a, align='center')
plt.xticks(age)
ax.yaxis.grid(True, which='major')
plt.ylabel('Count')
plt.xlabel('Age')
plt.show()

#### Ethnicity

In [ ]:
ethnicity = subject_df["ethnicity"].unique()
count_e = [len(subject_df[subject_df["ethnicity"] == x]) for x in ethnicity]

fig, ax = plt.subplots()
ax.pie(count_e, labels=ethnicity)
ax.set_title("Ethnicity")
plt.show()

### EDA analisis

#### id --> numero del pacient

In [ ]:
patients_number = [int(x.split('_')[1]) for x in glob("Patient_*", root_dir=path_patients)]
id2num = {}
for n in patients_number:
    try:
        with open(path_patients / f"Patient_{n}/meta_{n}.txt") as meta:
            id = int(meta.readline().split(',')[1][:-1])
        id2num[id] = n
    except:
        print(f"La carpeta Patient_{n} no conte el archiu meta_{n}.txt")

id2num


#### per a un pacient nomes

In [ ]:
id = 19
path_pat = path_patients / f"Patient_{id2num[id]}"
df_signal = open_signals_npy(path_pat)
df_signal

In [ ]:
id = 2
path_pat = path_patients / f"Patient_{id2num[id]}"
df_signal = open_signals_npy(path_pat)
fig, ax = plt.subplots()
ax.plot(df_signal['time'], df_signal['eda'])
plt.ylabel('EDA')
plt.xlabel('Time')
ax.set_title(f"Id: {id}")
plt.show()

In [ ]:
fig, axs = plt.subplots(2)
fig.set_size_inches(12,7)
plt.subplots_adjust(hspace=0.35)

id = 42
path_pat = path_patients / f"Patient_{id2num[id]}"
df_signal = open_signals_npy(path_pat)
df_signal['timestamp'] = df_signal.time.apply(lambda y : y.timestamp())
df_signal['timestamp'] = df_signal['timestamp'] - df_signal['timestamp'][0]

axs[0].plot(df_signal['timestamp'], df_signal['eda'])
axs[0].set_ylabel('EDA')
axs[0].set_xlabel('Temps [s]')


id = 54
path_pat = path_patients / f"Patient_{id2num[id]}"
df_signal = open_signals_npy(path_pat)
df_signal['timestamp'] = df_signal.time.apply(lambda y : y.timestamp())
df_signal['timestamp'] = df_signal['timestamp'] - df_signal['timestamp'][0]

axs[1].plot(df_signal['timestamp'], df_signal['eda'])
axs[1].set_ylabel('EDA')
axs[1].set_xlabel('Temps [s]')
plt.show()

#### Per a tots els pacients

In [ ]:
n = int(np.ceil(np.sqrt(len(id2num))))
#print(len(id2num),n)
#print(n,n if n*n==len(id2num) else n+1)
ids_list = sorted([i for i in id2num])
m = int(np.ceil(len(id2num)/n))
#print(n,m)

fig, axs = plt.subplots(n,m)
fig.set_size_inches(2*m,2*n)
#axs.axis('off')
for i, id in enumerate(ids_list):
    df_signal = open_signals_npy(path_patients / f"Patient_{id2num[id]}")
    pos = (i//m,i%m)
    axs[pos[0],pos[1]].plot(df_signal['time'], df_signal['eda'])
    axs[pos[0],pos[1]].set_title(f"Id: {id}")
    axs[pos[0],pos[1]].set_xticklabels([])
    axs[pos[0],pos[1]].set_yticklabels([])

for i in range(n*m-len(id2num)):
    pos = (n-1,m-i-1)
    axs[pos[0],pos[1]].set_xticklabels([])
    axs[pos[0],pos[1]].set_yticklabels([])

#axs[n//2,0].ylabel('EDA')
#axs[0,m//2].xlabel('Time')
plt.savefig(path_img_save / "EDA Grafics.jpg", bbox_inches='tight')
plt.show()

#### Grafic rPPG

In [ ]:
fig, axs = plt.subplots(2)
fig.set_size_inches(6,7)
plt.subplots_adjust(hspace=0.35)

id = 42
path_pat = path_patients / f"Patient_{id2num[id]}"
df_signal = open_signals_npy(path_pat)
df_signal['timestamp'] = df_signal.time.apply(lambda y : y.timestamp())
df_signal['timestamp'] = df_signal['timestamp'] - df_signal['timestamp'][0]


img_path = path_pat / 'imgs'
bvp_signal, wang_signal, wang_signal_raw = rPPG_detector(img_path,df_signal,
                                                         fps=30, time=5)

axs[0].plot(np.array(range(len(bvp_signal)))/30, bvp_signal, label='PPG_signal')
axs[0].plot(np.array(range(len(bvp_signal)))/30, wang_signal, label='rPPG_signal')
#axs[0].plot(wang_signal_raw, label='rPPG_signal_POS_raw')
axs[0].set_ylabel('PPG')
axs[0].set_xlabel('Temps [s]')
axs[0].legend(loc='upper right')


id = 54
path_pat = path_patients / f"Patient_{id2num[id]}"
df_signal = open_signals_npy(path_pat)
df_signal['timestamp'] = df_signal.time.apply(lambda y : y.timestamp())
df_signal['timestamp'] = df_signal['timestamp'] - df_signal['timestamp'][0]

img_path = path_pat / 'imgs'
bvp_signal, wang_signal, wang_signal_raw = rPPG_detector(img_path,df_signal,
                                                         fps=30, time=5)

axs[1].plot(np.array(range(len(bvp_signal)))/30, bvp_signal, label='PPG_signal')
axs[1].plot(np.array(range(len(bvp_signal)))/30, wang_signal, label='rPPG_signal')
#axs[1].plot(wang_signal_raw, label='rPPG_signal_POS_raw')
axs[1].set_ylabel('PPG')
axs[1].set_xlabel('Temps [s]')
#axs[1].legend()
plt.show()

In [ ]:
n = int(np.ceil(np.sqrt(len(id2num))))
#print(len(id2num),n)
#print(n,n if n*n==len(id2num) else n+1)
ids_list = sorted([i for i in id2num])
m = int(np.ceil(len(id2num)/n))
#print(n,m)

fig, axs = plt.subplots(n,m)
fig.set_size_inches(2*m,2*n)
for i, id in tqdm(enumerate(ids_list)):
    dataset_path= path_patients / f'Patient_{id2num[id]}'
    img_path = dataset_path / 'imgs'
    df_bio = open_signals_npy(dataset_path)
    bvp_signal, wang_signal, wang_signal_raw = rPPG_detector(img_path,df_bio,
                                                             fps=30,
                                                             time=5)
    pos = (i//m,i%m)
    axs[pos[0],pos[1]].plot(bvp_signal, label='PPG_signal')
    axs[pos[0],pos[1]].plot(wang_signal, label='rPPG_signal')
    axs[pos[0],pos[1]].plot(wang_signal_raw, label='rPPG_signal_POS_raw')
    #axs[pos[0],pos[1]].legend()
    axs[pos[0],pos[1]].set_title(f"Id: {id}")
    axs[pos[0],pos[1]].set_xticklabels([])
    axs[pos[0],pos[1]].set_yticklabels([])

for i in range(n*m-len(id2num)):
    pos = (n-1,m-i-1)
    axs[pos[0],pos[1]].set_xticklabels([])
    axs[pos[0],pos[1]].set_yticklabels([])

plt.savefig(path_img_save / "rPPG Grafics.jpg", bbox_inches='tight')
plt.show()

In [ ]:
df = pd.read_csv("C:/Users/Xavi/Desktop/Dataset_init/id_bbox.csv")

if 10 not in list(df.patient):
    print(1)


In [ ]:
N = 10

n = int(np.ceil(np.sqrt(N)))
m = int(np.ceil(N/n))

print(n,m)
for i in range(N):
    pos = (i//m,i%m)
    print(pos)
print("--------")
for i in range(n*m-N):
    pos = (n-1,m-i-1)
    print(pos)

In [ ]:
t = 10
fps=30

for id in tqdm(id2num):
    imgs_dir= path_patients / f'Patient_{id2num[id]}/imgs'
    img_names = [_ for _ in os.listdir(imgs_dir) if '.png' in _]
    pad = len(str(len(img_names)))
    img_names.sort(key=lambda x: str(x.split('.')[0]).zfill(pad))
    video = []
    for i in range(0,len(img_names),t*60): # 60 == total fps
        pic = imageio.imread(imgs_dir / img_names[i])
        video.append(pic)

    imageio.mimsave(path_img_save / f"gifs/imgs_{id}.gif", video, duration=1/fps)

In [ ]:
t = 5
fps=30

id = 19

imgs_dir= path_patients / f'Patient_{id2num[id]}/imgs'
img_names = [_ for _ in os.listdir(imgs_dir) if '.png' in _]
pad = len(str(len(img_names)))
img_names.sort(key=lambda x: str(x.split('.')[0]).zfill(pad))
video = []
for i in range(0,len(img_names),t*60): # 60 == total fps
    pic = imageio.imread(imgs_dir / img_names[i])
    video.append(pic)

imageio.mimsave(path_img_save / f"gifs/imgs_{id}.gif", video, duration=1/fps)

In [ ]:
t = 15
fps=30

ids_list = sorted([i for i in id2num])
t_img = t*fps

list_imgs = []
for id in tqdm(ids_list):
    imgs_dir= path_patients / f'Patient_{id2num[id]}/imgs'
    img_names = [_ for _ in os.listdir(imgs_dir) if '.png' in _]
    pad = len(str(len(img_names)))
    img_names.sort(key=lambda x: str(x.split('.')[0]).zfill(pad))

    imgs = []
    skip = len(img_names)//t_img
    for i in range(0,t_img):
        pic = imageio.imread(imgs_dir / img_names[i*skip])
        imgs.append(pic)
    
    list_imgs.append(imgs)

n = int(np.ceil(np.sqrt(len(id2num))))
m = int(np.ceil(len(id2num)/n))
video = []
for i in tqdm(range(0,t_img)):
    for j in n:
        for k in m:
            

In [ ]:
t = 15
fps=30

ids_list = sorted([i for i in id2num])

for id in ids_list:
    imgs_dir= path_patients / f'Patient_{id2num[id]}/imgs'
    img_names = [_ for _ in os.listdir(imgs_dir) if '.png' in _]
    pad = len(str(len(img_names)))
    img_names.sort(key=lambda x: str(x.split('.')[0]).zfill(pad))
    cont = 0
    skip = len(img_names)//(t*fps)
    for i in range(0,t*fps):
        cont += 1
    print("id:"+str(id), "cont:"+str(cont))

In [ ]:
t = 15
fps=30

ids_list = [53,54,55,56,57,58,59,60,61,62,63,64]
for id in tqdm(ids_list):
    imgs_dir= path_patients / f'Patient_{id2num[id]}/imgs'
    img_names = [_ for _ in os.listdir(imgs_dir) if '.png' in _]
    pad = len(str(len(img_names)))
    img_names.sort(key=lambda x: str(x.split('.')[0]).zfill(pad))
    video = []
    for i in range(0,len(img_names),t*60): # 60 == total fps
        pic = imageio.imread(imgs_dir / img_names[i])
        video.append(pic)

    imageio.mimsave(path_img_save / f"gifs/imgs_{id}.gif", video, duration=1/fps)

In [ ]:
gifs_folder = path_img_save / "gifs"
archivos = os.listdir(gifs_folder)


with open(gifs_folder / archivos[0],'rb') as f:
    display(Image(data=f.read(), format='png',width=512,height=512))